In [1]:
from pymilvus import connections, Collection

connections.connect(uri="http://localhost:19530")  # Replace with your Milvus server IP

In [2]:
collection = Collection(name="bat_dong_san")
collection.load()

## Search


In [3]:
from pymilvus import AnnSearchRequest
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from pymilvus import WeightedRanker

rerank = WeightedRanker(0.2, 0.8)
bge_m3_ef = BGEM3EmbeddingFunction(use_fp16=False, device="cpu")

c:\Users\daoqu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 29255.78it/s]
c:\Users\daoqu\AppData\Local\Programs\Python\Python310\lib\site-packages\FlagEmbedding\BGE_M3\modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub f

In [4]:
def query(text, limit=3):
    query_embeds = bge_m3_ef.encode_queries([text])

    content_sparse_search_params = {
        "data": query_embeds["sparse"],
        "anns_field": "content_sparse",
        "param": {
            "metric_type": "IP",
        },
        "limit": limit,
    }
    content_sparse_request = AnnSearchRequest(**content_sparse_search_params)

    content_dense_search_params = {
        "data": query_embeds["dense"],
        "anns_field": "content_dense",
        "param": {
            "metric_type": "IP",
            "ef": 20,
        },
        "limit": limit,
    }
    content_dense_request = AnnSearchRequest(**content_dense_search_params)

    reqs = [
        content_sparse_request,
        content_dense_request,
    ]

    res = collection.hybrid_search(
        reqs,  # List of AnnSearchRequests created in step 1
        rerank,  # Reranking strategy specified in step 2
        limit=limit,  # Number of final search results to return
        output_fields=["id", "url", "title_text", "content_text"],
    )
    return res

In [5]:
results = query(
    "Căn cứ vào Điều 1 của Quyết định 806/QĐ-UBND, UBND huyện Mường Ảng có trách nhiệm gì?"
)

# print(results)
for i in results[0]:
    print(i.id, i.score, i.title_text)

97930001 0.5485317707061768 Quyết định 289/QĐ-UBND phê duyệt Kế hoạch sử dụng đất năm 2018 của huyện Mường Ảng do tỉnh Điện Biên ban hành
124280001 0.5457984805107117 Quyết định 562/QĐ-UBND phê duyệt kế hoạch sử dụng đất năm 2016 huyện Mường Ảng tỉnh Điện Biên
85690002 0.5450834631919861 Quyết định 366/QĐ-UBND về phê duyệt Kế hoạch sử dụng đất năm 2019 của huyện Mường Ảng tỉnh Điện Biên


In [6]:
for i in results[0]:
    print(i.content_text)
    print("-" * 100)
    print()
    print()

Chunk header:
Document title: Quyết định 289/QĐ-UBND phê duyệt Kế hoạch sử dụng đất năm 2018 của huyện Mường Ảng do tỉnh Điện Biên ban hành 
Chunk text:
ỦY
  BAN NHÂN DÂN
         


          TỈNH ĐIỆN BI
         
         Ê
        


          N
          
          -------
         

          CỘNG
  HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
          
          Độc lập - Tự do - Hạnh phúc
          
          ---------------
         


         Số:
        
        289
        
         /QĐ-UBND
        


          Điện
  Biên, ngày
         
         13
        


          tháng
         
         0
        


          4
  năm 2018
         

        QUYẾT ĐỊNH
       


       VỀ VIỆC PHÊ DUYỆT KẾ HOẠCH SỬ DỤNG ĐẤT NĂM 2018 CỦA HUYỆN MƯỜNG ẢNG
      

       ỦY BAN NHÂN DÂN TỈNH ĐIỆN BIÊN
      

       Căn cứ Luật Tổ chức Chính quyền địa
phương ngày 19 tháng 6 năm 2015;
      

       Căn cứ Luật Đất đai ngày 29 tháng
11 năm 2013;
      

       Căn cứ Nghị định số 43/2014/NĐ-CP ngày
